In [1]:
from __future__ import absolute_import
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import numpy as np
import random
import math
from preprocess import get_data
from sklearn import preprocessing
from main import *

2023-05-02 18:18:20.909199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Get data and reshape

#Import and reshape
X_train, X_test, X_val, Y_train, Y_test, Y_val = get_data(.3, "./../barcoded_data/")

char_encoder = preprocessing.LabelEncoder().fit(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1))))
orig_shapes = [Y_train.shape[0], Y_test.shape[0], Y_val.shape[0]]
Y_train = char_encoder.transform(Y_train.reshape(-1)).reshape((orig_shapes[0], 4))
Y_test = char_encoder.transform(Y_test.reshape(-1)).reshape((orig_shapes[1], 4))
Y_val = char_encoder.transform(Y_val.reshape(-1)).reshape((orig_shapes[2], 4))

#Convert input to properly shaped and typed tensors
X_train = tf.convert_to_tensor(np.asarray(np.reshape(X_train, (-1, *X_train.shape[-2:]))).astype(np.float32))
X_val   = tf.convert_to_tensor(np.asarray(np.reshape(X_val  , (-1, *X_val.shape[-2:]))).astype(np.float32))
Y_train = tf.convert_to_tensor(np.asarray(np.reshape(Y_train, (-1))))
Y_val   = tf.convert_to_tensor(np.asarray(np.reshape(Y_val  , (-1))))

#Don't reshape test, because we need to preserve CAPTCHAs but convert to tensor
X_test  = tf.convert_to_tensor(np.asarray(X_test).astype(np.float32))
Y_test  = tf.convert_to_tensor(np.asarray(Y_test))

#Expand Dims
X_train = tf.expand_dims(X_train, axis=-1)
X_val   = tf.expand_dims(X_val  , axis=-1)

#One hot encode
Y_val   = tf.one_hot(Y_val  , depth=31, axis=-1)
Y_train = tf.one_hot(Y_train, depth=31, axis=-1)

In [7]:
#Fit and save model
model = Model()
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(.0001),
                metrics= [
                tf.keras.metrics.Accuracy()
                ])
model.fit(
    X_train,
    Y_train,
    epochs=3,
    batch_size=256,
    validation_data=(X_val, Y_val)
)

model.save('./../models/barcoded_1')

Epoch 1/3
109/109 [==============================] - 42s 370ms/step - loss: 3.4700 - categorical_crossentropy: 3.4700 - val_loss: 3.4020 - val_categorical_crossentropy: 3.4020
Epoch 2/3
109/109 [==============================] - 44s 403ms/step - loss: 2.9834 - categorical_crossentropy: 2.9834 - val_loss: 3.1430 - val_categorical_crossentropy: 3.1430
Epoch 3/3
109/109 [==============================] - 48s 442ms/step - loss: 2.6818 - categorical_crossentropy: 2.6818 - val_loss: 2.7110 - val_categorical_crossentropy: 2.7110


INFO:tensorflow:Assets written to: ./../models/barcoded_1/assets


INFO:tensorflow:Assets written to: ./../models/barcoded_1/assets


In [10]:
#Load a model and run tests to get accuracy and print an example
model = tf.keras.models.load_model("./../models/segmented")
#TODO: I don't know how to properly load and save models, it's not working

def get_acc(X_test, Y_test):
    X_test    = np.reshape(X_test, (-1, *X_test.shape[-2:]))
    Y_test    = np.reshape(Y_test, (-1))
    
    probs     = model.predict(X_test, verbose=0)
    diff      = np.argmax(probs, axis=1) - Y_test
    char_acc  = np.where(diff == 0, 1, 0)

    captcha_acc = np.array_split(diff, diff.shape[0]//4)
    captcha_acc = np.apply_along_axis((lambda x: 1 if np.all(x == 0) else 0), axis=1, arr=captcha_acc)

    return np.mean(char_acc), np.mean(captcha_acc)
    

output = model.predict(X_test[7], verbose=0)

print("Secret CAPTCHA:")
print(char_encoder.inverse_transform(Y_test[7]))

print("Model guess:")
print(char_encoder.inverse_transform(np.argmax(output, axis=1)))

char_acc, captcha_acc = get_acc(X_test, Y_test)
print(f"Character accuracy: {round(100*char_acc, 2)}% \nCaptcha accuracy: {round(100*captcha_acc, 2)}%")


NotImplementedError: in user code:

    File "/Users/mahlonpage/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Users/mahlonpage/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/mahlonpage/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/mahlonpage/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Users/mahlonpage/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None